___
___
# **Proyecto Final Data Science**
#### **Pipeline NLP | Similitud por atributos del producto.**
Equipo: 2 - Quantum Insights
Integrantes:
- Felipe Varela - Product Owner
- Freddy Yaquive - Data Scientist
- Ivan Martinez - Data Scientist
- Sebastian Moya - Data Scientist
- Nicolás Lazarte - Scrum Master

Cohorte: DSFT01
___


- Este modelo actúa como un experto en catálogo. No necesita saber quién es el usuario ni qué ha comprado antes el resto de la gente; solo se fija en las características intrínsecas del producto.

**¿Cómo funciona?**

- Vectorización (TF-IDF): Toma la información textual del producto (Nombre, Marca, Categoría) y la convierte en números. Da más peso a las palabras que son únicas y distintivas (como "Orgánico" o "Sin Gluten") y menos peso a palabras comunes.

- Cálculo de Similitud (Coseno): Mide la distancia matemática entre los productos. Si dos productos tienen vectores que apuntan a la misma dirección, significa que hablan de lo mismo.

**Caso de uso ideal:**

- Cuando un usuario está viendo un producto y queremos mostrarle: "Productos similares a este".

- Soluciona el problema de Cold Start (Arranque en frío): Puede recomendar productos nuevos que nadie ha comprado aún, simplemente analizando su descripción.


___
___

In [102]:
# Librerías
import pandas as pd
import os
import pickle # para guardar archivos binarios (el cerebro guardado)
from sklearn.feature_extraction.text import TfidfVectorizer # para convertir texto en vectores númericos
from sklearn.metrics.pairwise import linear_kernel # para medir la similitud entre productos


___
### **1. Carga y Preparación de Datos**

In [103]:
# Definimos una función para limpiar y preparar
def cargar_y_preparar_datos(ruta_archivo):
    try: 
        # Intentar cargar el archivo CSV con los datos
        df = pd.read_csv(ruta_archivo)
    except FileNotFoundError:
        return None, "Archivo no encontrado"

    # Definir las columnas necesarias para crear el perfil
    columnas_clave = ['ProductName', 'Brand', 'Category', 'SubCategory']
    
    for col in columnas_clave:
        # Rellenar valores vacíos para evitar errores al unir texto
        df[col] = df[col].fillna('') 

    # Función auxiliar para crear un perfil de texto único por producto
    def unir_columnas(fila):
        # Concatenar marca, categoría y nombre para dar contexto al modelo
        return f"{fila['Brand']} {fila['Category']} {fila['SubCategory']} {fila['ProductName']}"

    # Aplicar la función de unión a cada fila del conjunto de datos
    df['perfil_texto'] = df.apply(unir_columnas, axis=1)
    
    # Normalizar el texto a minúsculas y eliminar espacios sobrantes
    df['perfil_texto'] = df['perfil_texto'].str.lower().str.strip()     
    
    print("Datos listos. 'perfil_texto'.")
    print("---"*10)
    return df

# Ejecutar la función para cargar los datos
df_productos = cargar_y_preparar_datos("../databases/products.csv")

# Visualizar una muestra del texto procesado
print(f"\nPerfil creado: \n{df_productos['perfil_texto'].iloc[0:5]}")

Datos listos. 'perfil_texto'.
------------------------------

Perfil creado: 
0    fresho fruits & vegetables potato, onion & tom...
1    fresho fruits & vegetables potato, onion & tom...
2    fresho eggs, meat & fish farm eggs farm eggs -...
3    fresho fruits & vegetables potato, onion & tom...
4    fresho fruits & vegetables potato, onion & tom...
Name: perfil_texto, dtype: object


___
### **2. Entrenamiento del Pipeline**

In [104]:
# Función para entrenar el pipeline
def entrenar_pipeline(df):
    print("1. Iniciando vectorización TF-IDF...")
    
    # Inicializar la herramienta para convertir texto en vectores numéricos
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Transformar el texto de los productos en una matriz matemática
    matriz_tfidf = tfidf.fit_transform(df['perfil_texto'])
    print(f"   Matriz creada: {matriz_tfidf.shape} (Productos x Palabras)")
    print("---"*10)

    print("2. Calculando similitud del coseno...")
    # Calcular la similitud matemática entre todos los pares de productos
    matriz_similitud = linear_kernel(matriz_tfidf, matriz_tfidf)
    print("---"*10)

    print("3. Guardando modelo...")
    
    
    # Construir la ruta subiendo un nivel (..) hacia la carpeta existente
    nombre_archivo = os.path.join('..', 'modelos_entrenados', 'modelo_recomendacion_npl.pkl')
    
    # Imprimir la ruta absoluta para verificar dónde se guardará
    print(f"   Ruta destino: {os.path.abspath(nombre_archivo)}")

    # Abrir el archivo en modo escritura binaria y guardar la matriz
    with open(nombre_archivo, 'wb') as f:
        pickle.dump(matriz_similitud, f)
        
    print(f"✅ Modelo guardado exitosamente.")
    
    return matriz_similitud

# Ejecutar el entrenamiento y almacenar el resultado en memoria
matriz_entrenada = entrenar_pipeline(df_productos)

1. Iniciando vectorización TF-IDF...
   Matriz creada: (2000, 2636) (Productos x Palabras)
------------------------------
2. Calculando similitud del coseno...
------------------------------
3. Guardando modelo...
   Ruta destino: e:\_Cursos\Tareas_Proyectos_y_Notas\Soy Henry\ProyectoFinal\PF-Quantum_Insights\modelos_entrenados\modelo_recomendacion_npl.pkl
✅ Modelo guardado exitosamente.


___
### **3. Uso del Modelo (Lógica de Recomendación)**

In [105]:
# Función para obtener recomendaciones
def obtener_recomendaciones(nombre_producto, df, matriz_similitud):
    print(f"\nRecomendaciones para: '{nombre_producto}'...")
    
    # Crear un índice para localizar rápidamente la fila de cada producto por su nombre
    indices = pd.Series(df.index, index=df['ProductName']).drop_duplicates()

    # Validar si el producto solicitado existe en la base de datos
    if nombre_producto not in indices:
        return "El producto no existe, intente con otro producto."

    # Obtener la posición numérica del producto en la matriz
    idx = indices[nombre_producto]

    # Obtener y listar los puntajes de similitud de este producto contra todos los demás
    puntajes = list(enumerate(matriz_similitud[idx]))

    # Ordenar los puntajes de mayor a menor similitud
    puntajes = sorted(puntajes, key=lambda x: x[1], reverse=True)

    # Seleccionar los 3 mejores resultados (excluyendo el primero que es el producto mismo)
    top_3 = puntajes[1:4]
    
    # Extraer los índices numéricos de los productos seleccionados
    indices_recomendados = [i[0] for i in top_3]
    
    # Recuperar la información detallada de los productos recomendados
    resultado = df[['ProductName', 'Brand', 'Category']].iloc[indices_recomendados]
    
    return resultado

___
#### **4. Ejecución y Prueba de pipeline**

In [106]:
# Guardamos solo la columna de nombres en un archivo CSV nuevo, para tener la lista de productos completa
# df_productos['ProductName'].to_csv('../databases/lista_productos_completa.csv', index=False)

In [107]:
# Muestra 20 nombres de productos al azar para realizar la prueba
print("Productos disponibles (aleatorio)")
muestra = df_productos['ProductName'].sample(20).values
for producto in muestra:
    print(f"- {producto}")

Productos disponibles (aleatorio)
- Detergent Washing Powder - Matic Front Load
- Sunrise Instant Coffee Powder - Chicory Mix, Extra Strong
- Aqua Gel - Naturale Aloe, 9 to 5
- Chicken Drumstick 1kg +Peri Peri Chicken Wings 500g
- Wonder Diaper Pants - Medium, 7-12 Kg
- Signature - Mysterious, Long Lasting, No Gas, Deodorant Bodyspray, Perfume For Men
- Maxfresh - Plax Pepper Mint, Alcohol Free
- Fresh Guard Disinfectant Toilet Cleaner Liquid, Lime Fresh
- Mini Samosa
- Toothpaste - Strong Teeth, Dental Cream, Anti Cavity
- Intense Oil Clear Lemon Face Wash
- Mango - 100% Natural Cold Pressed Juice
- Dark Fantasy - Choco Fills
- Frozen Sweet Corn - Preservative Free, Fresh & Juicy Kernels
- Mustard/Sasive - Small
- Italian Spice Mix
- Soft-Light Moisturising Cream
- Ceramic Tea/Coffee/Milk Mug - Yellow & Orange Design, Hand Painted
- Organic - Ragi Flour/Ragi Hittu
- Stainless Steel Flat Bottom Tope/Patila/Bhagona - No. 14


In [108]:
# Buscador
palabra_clave = "Chia Seeds" 

# Filtrar productos que contengan la palabra clave ignorando mayúsculas
resultados = df_productos[df_productos['ProductName'].str.contains(palabra_clave, case=False)]['ProductName'].unique()

print(f"Productos encontrados con '{palabra_clave}': {len(resultados)}")
print("---"*10)
for p in resultados[:10]: 
    print(f"- {p}")


print("---"*10)

# Ejemplo: si encontraste 'Green Tea' en la búsqueda
producto_seleccionado = 'Black Chia Seeds' 

recomendaciones = obtener_recomendaciones(producto_seleccionado, df_productos, matriz_entrenada)
display(recomendaciones)

Productos encontrados con 'Chia Seeds': 3
------------------------------
- Black Chia Seeds
- Natives Chia Seeds
- Chia Seeds
------------------------------

Recomendaciones para: 'Black Chia Seeds'...


,ProductName,Brand,Category
1080,Dry Fruits & Nuts,Fresho Signature,Gourmet & World Food
1079,Fruits & Nuts,Fresho Signature,Gourmet & World Food
1277,Strawberry,Fresho Signature,Gourmet & World Food


___

In [109]:
# Prueba 2

# Elegimos un producto al azar para probar
producto_ejemplo = df_productos['ProductName'].iloc[100] # <-- cambia el producto  


# Llamamos a la función pasando el producto, la base de datos y la matriz matemática
recomendaciones = obtener_recomendaciones(producto_ejemplo, df_productos, matriz_entrenada)
print("---"*10)
print("\nRecomendaciones:")
display(recomendaciones) 


Recomendaciones para: 'Marvel 20 Multi Storage Plastic Basket Without Lid - Matt Brown'...
------------------------------

Recomendaciones:


,ProductName,Brand,Category
315,Magic Seal Square Plastic Storage Containers -...,Polyset,"Kitchen, Garden & Pets"
549,Magic Seal Round Plastic Oil Containers - Roya...,Polyset,"Kitchen, Garden & Pets"
1099,"Premium Storage Container with Lid - Pink, Pla...",Mastercook,"Kitchen, Garden & Pets"


___
### **Evaluación del Modelo**
___

In [110]:
# --- EVALUACIÓN DEL MODELO NLP ---

def evaluar_confianza_y_cobertura(matriz_sim, df_prods, n_muestras=1000):
    print(f"Evaluando con una muestra de {n_muestras} productos...")
    
    # Seleccionar productos al azar para la prueba
    indices_muestra = np.random.choice(matriz_sim.shape[0], n_muestras, replace=False)
    
    scores_promedio = []
    
    # Revisar las recomendaciones para cada producto de la muestra
    for idx in indices_muestra:
        # Obtener los puntajes de similitud de esa fila
        puntajes = matriz_sim[idx]
        
        # Ordenar y tomar el Top 3 (excluyendo el propio producto que es el índice 0)
        # argsort ordena de menor a mayor, tomamos los últimos (mayores)
        top_indices = puntajes.argsort()[-(3+1):-1][::-1]
        
        # Obtener los valores de similitud de esos 3 mejores
        top_scores = puntajes[top_indices]
        
        # Guardar el promedio de similitud de estas 3 recomendaciones
        scores_promedio.append(np.mean(top_scores))
        
    media_confianza = np.mean(scores_promedio)
    
    print(f"✅ Confianza Promedio del Modelo: {media_confianza:.4f} (Escala 0 a 1)")
    print("Interpretación: \n- Cercano a 1: El modelo encuentra productos casi idénticos (muy preciso).\n- Cercano a 0: El modelo no encuentra buenas similitudes (recomendaciones débiles).")
    
    return media_confianza

# Necesitamos numpy para los cálculos aleatorios
import numpy as np

# Ejecutar la evaluación usando la matriz que ya entrenaste
evaluar_confianza_y_cobertura(matriz_entrenada, df_productos)

Evaluando con una muestra de 1000 productos...
✅ Confianza Promedio del Modelo: 0.6628 (Escala 0 a 1)
Interpretación: 
- Cercano a 1: El modelo encuentra productos casi idénticos (muy preciso).
- Cercano a 0: El modelo no encuentra buenas similitudes (recomendaciones débiles).


np.float64(0.6628099810193857)